In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import shutil

!unrar x '/content/drive/MyDrive/forest_fingerprint/training.rar'


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal

Cannot open /content/drive/MyDrive/forest_fingerprint/trainingfinal5.rar
No such file or directory
No files to extract


In [ ]:
import shutil
from pathlib import Path
from fastai.vision.all import *
import gc
import torch
path = Path('/content/training')
class_dirs = [path / f"{prefix}{i}" for prefix in ["b", "p","c"] for i in range(1, 7)]
def label_func(x):
    return x.parts[-2][0]
# Function to move a folder temporarily
def move_folder_temporarily(src, dest):
    shutil.move(src, dest)
    return lambda: shutil.move(dest, src)

def get_image_files_without_temp(path):
    return [img for img in get_image_files(path) if "temp" not in str(img)]

# Training loop using leave-one-out approach
for i, test_dir in enumerate(class_dirs):
    print(f"Leave-one-out run {i+1}: using {test_dir.name} as test set")

    # Move the test directory temporarily
    restore_folder = move_folder_temporarily(test_dir, path / "temp")

    # Create DataLoaders
    dls = ImageDataLoaders.from_path_func(path, get_image_files_without_temp(path), label_func, valid_pct=0.25,batch_tfms=[*aug_transforms(), Normalize.from_stats(*imagenet_stats)])

    # Filter out the 'temp' folder from the classes
    dls.vocab = L(dls.vocab).filter(lambda x: x != 'temp')


    # Create a cnn_learner with your desired architecture
    from fastai.optimizer import Adam

    learn = vision_learner(dls, resnet34, metrics=accuracy)


# Train the model using fit_one_cycle
    lr = 0.007  # Replace with the learning rate you found
    learn.unfreeze()
    n_epochs = 15
    opt = Adam(learn.model.parameters(), lr=lr)
    learn.fit_one_cycle(n_epochs, lr)



    # Restore the test directory
    restore_folder()

    # Predict on the test set and save the predictions
    test_files = list(test_dir.glob("*.jpeg"))  # Adjust the file format as needed
    test_dl = learn.dls.test_dl(test_files)
    preds, _ = learn.get_preds(dl=test_dl)

    # Save the predictions for the current test set
    with open(f"predictions_{test_dir.name}.txt", "w") as f:
        for test_file, pred in zip(test_files, preds):
            predicted_class = learn.dls.vocab[np.argmax(pred)]
            f.write(f"{test_file}: {predicted_class}\n")

    del learn
    torch.cuda.empty_cache()
    gc.collect()